# 🏥 Crawler Industrial de Grupos WhatsApp Médicos

Extrai grupos de WhatsApp de todos os principais agregadores de vagas médicas do Brasil.

## Sites cobertos:
1. **Escala de Plantão** - HTML simples (requests)
2. **Grupos Médicos** - HTML simples (requests)
3. **Quero Plantão** - JavaScript (Playwright Async)
4. **Search Plantão** - JavaScript (Playwright Async)
5. **Plantões Médicos Brasil** - JavaScript (Playwright Async)

## Requisitos:
```bash
pip install requests beautifulsoup4 pandas lxml playwright nest-asyncio
playwright install chromium
```

In [ ]:
# Instalação (rodar apenas uma vez)
!pip install requests beautifulsoup4 pandas lxml playwright nest-asyncio -q
!playwright install chromium

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
import asyncio
from datetime import datetime
from playwright.async_api import async_playwright
import nest_asyncio

# Permite rodar asyncio dentro do Jupyter
nest_asyncio.apply()

print("✅ Bibliotecas carregadas")

✅ Bibliotecas carregadas


## 1. Funções Auxiliares

In [2]:
def extract_whatsapp_code(url):
    """Extrai o código do convite do link do WhatsApp"""
    match = re.search(r'chat\.whatsapp\.com/([A-Za-z0-9]+)', url)
    return match.group(1) if match else None

def extract_telegram_code(url):
    """Extrai o código/username do link do Telegram"""
    match = re.search(r't\.me/(?:joinchat/)?([A-Za-z0-9_-]+)', url)
    return match.group(1) if match else None

def detect_state(text):
    """Detecta o estado brasileiro no texto"""
    states = {
        'AC': ['acre'], 'AL': ['alagoas'], 'AP': ['amapá', 'amapa'],
        'AM': ['amazonas'], 'BA': ['bahia'], 'CE': ['ceará', 'ceara'],
        'DF': ['distrito federal', 'brasília', 'brasilia', ' df '],
        'ES': ['espírito santo', 'espirito santo'],
        'GO': ['goiás', 'goias'], 'MA': ['maranhão', 'maranhao'],
        'MT': ['mato grosso'], 'MS': ['mato grosso do sul'],
        'MG': ['minas gerais'], 'PA': ['pará'],
        'PB': ['paraíba', 'paraiba'], 'PR': ['paraná', 'parana'],
        'PE': ['pernambuco'], 'PI': ['piauí', 'piaui'],
        'RJ': ['rio de janeiro'], 'RN': ['rio grande do norte'],
        'RS': ['rio grande do sul'], 'RO': ['rondônia', 'rondonia'],
        'RR': ['roraima'], 'SC': ['santa catarina'],
        'SP': ['são paulo', 'sao paulo', ' sp ', 'interior-sp', 'interior sp'],
        'SE': ['sergipe'], 'TO': ['tocantins'],
        'NORTE': ['região norte', 'regiao norte'],
        'NORDESTE': ['região nordeste', 'regiao nordeste', 'nordeste'],
        'SUDESTE': ['região sudeste', 'regiao sudeste', 'sudeste'],
        'SUL': ['região sul', 'regiao sul'],
        'CENTRO_OESTE': ['centro-oeste', 'centro oeste']
    }
    
    text_lower = ' ' + text.lower() + ' '
    for state_code, patterns in states.items():
        for pattern in patterns:
            if pattern in text_lower:
                return state_code
    return 'BR'

def detect_category(text):
    """Detecta a categoria/especialidade"""
    categories = {
        'plantao_geral': ['plantão', 'plantao', 'vagas', 'emprego', 'escala'],
        'cardiologia': ['cardiologia', 'cardiologista', 'cardio'],
        'pediatria': ['pediatria', 'pediatra', 'pediátrico', 'neonat', 'neo '],
        'ginecologia': ['ginecologia', 'obstetrícia', 'obstetricia', 'gineco'],
        'ortopedia': ['ortopedia', 'ortopedista'],
        'neurologia': ['neurologia', 'neurologista', 'neuro'],
        'psiquiatria': ['psiquiatria', 'psiquiatra'],
        'anestesiologia': ['anestesiologia', 'anestesista', 'anestesia'],
        'cirurgia': ['cirurgia', 'cirurgião', 'cirurgiao'],
        'dermatologia': ['dermatologia', 'dermatologista'],
        'oftalmologia': ['oftalmologia', 'oftalmologista', 'oftalmo'],
        'urologia': ['urologia', 'urologista'],
        'emergencia': ['emergência', 'emergencia', 'urgência', 'urgencia', 'uti', 'intensiva'],
        'oncologia': ['oncologia', 'oncologista'],
        'gastroenterologia': ['gastroenterologia', 'gastro'],
        'pneumologia': ['pneumologia', 'pneumologista'],
        'infectologia': ['infectologia', 'infectologista'],
        'nefrologia': ['nefrologia', 'nefrologista'],
        'endocrinologia': ['endocrinologia', 'endocrinologista'],
        'reumatologia': ['reumatologia', 'reumatologista'],
        'geriatria': ['geriatria', 'geriatra'],
        'otorrino': ['otorrino', 'otorrinolaringologia'],
        'radiologia': ['radiologia', 'radiologista', 'ultrassonografia', 'ecografia'],
        'medicina_trabalho': ['trabalho', 'ocupacional'],
        'telemedicina': ['telemedicina', 'teleconsulta'],
        'medicina_familia': ['família', 'familia', 'comunidade'],
        'enfermagem': ['enfermagem', 'enfermeiro', 'enfermeira'],
        'odontologia': ['odontologia', 'dentista', 'odonto'],
        'fisioterapia': ['fisioterapia', 'fisioterapeuta'],
        'nutricao': ['nutrição', 'nutricao', 'nutricionista', 'nutrologia'],
        'psicologia': ['psicologia', 'psicólogo', 'psicologa'],
        'fonoaudiologia': ['fonoaudiologia', 'fonoaudiólogo'],
        'material_estudo': ['material', 'resumo', 'livro', 'artigo', 'pdf'],
        'discussao_casos': ['discussão', 'casos clínicos', 'ecg', 'eco'],
        'gestao': ['gestão', 'gestao', 'empreendedorismo'],
        'concurso_residencia': ['concurso', 'residência', 'residencia']
    }
    
    text_lower = text.lower()
    for category, patterns in categories.items():
        for pattern in patterns:
            if pattern in text_lower:
                return category
    return 'medicina_geral'

print("✅ Funções auxiliares carregadas")

✅ Funções auxiliares carregadas


## 2. Crawler com Requests (sites HTML simples)

In [3]:
def crawl_with_requests(url, source_name):
    """Crawler para sites que não usam JavaScript"""
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers, timeout=30)
        response.raise_for_status()
    except Exception as e:
        print(f"   ❌ Erro ao acessar {url}: {e}")
        return []
    
    soup = BeautifulSoup(response.text, 'lxml')
    groups = []
    
    # WhatsApp
    for link in soup.find_all('a', href=re.compile(r'chat\.whatsapp\.com')):
        href = link.get('href')
        name = link.get_text(strip=True) or 'Sem nome'
        
        # Tenta pegar contexto do elemento pai
        parent = link.find_parent(['div', 'section', 'article', 'li'])
        if parent:
            title = parent.find(['h2', 'h3', 'h4', 'h5', 'strong', 'b'])
            if title:
                name = title.get_text(strip=True)
        
        invite_code = extract_whatsapp_code(href)
        if invite_code and len(name) > 2:
            groups.append({
                'source': source_name,
                'platform': 'whatsapp',
                'name': name[:100],
                'url': href,
                'invite_code': invite_code,
                'state': detect_state(name),
                'category': detect_category(name),
                'scraped_at': datetime.now().isoformat()
            })
    
    # Telegram
    for link in soup.find_all('a', href=re.compile(r't\.me')):
        href = link.get('href')
        name = link.get_text(strip=True) or 'Sem nome'
        
        invite_code = extract_telegram_code(href)
        if invite_code and len(name) > 2:
            groups.append({
                'source': source_name,
                'platform': 'telegram',
                'name': name[:100],
                'url': href,
                'invite_code': invite_code,
                'state': detect_state(name),
                'category': detect_category(name),
                'scraped_at': datetime.now().isoformat()
            })
    
    return groups

print("✅ Crawler requests carregado")

✅ Crawler requests carregado


## 3. Crawler com Playwright ASYNC (sites JavaScript)

In [4]:
async def crawl_with_playwright_async(url, source_name, wait_time=5000, scroll=True):
    """Crawler ASYNC para sites que carregam conteúdo via JavaScript"""
    
    groups = []
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
        )
        page = await context.new_page()
        
        try:
            print(f"   Acessando {url}...")
            await page.goto(url, timeout=60000)
            
            # Espera inicial
            await page.wait_for_timeout(wait_time)
            
            # Scroll para carregar conteúdo lazy-loaded
            if scroll:
                for _ in range(5):
                    await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
                    await page.wait_for_timeout(1000)
            
            # Extrai links WhatsApp
            whatsapp_links = await page.query_selector_all('a[href*="chat.whatsapp.com"]')
            print(f"   Encontrados {len(whatsapp_links)} links WhatsApp")
            
            for link in whatsapp_links:
                try:
                    href = await link.get_attribute('href')
                    name = await link.inner_text()
                    name = name.strip() if name else 'Sem nome'
                    
                    invite_code = extract_whatsapp_code(href)
                    if invite_code:
                        groups.append({
                            'source': source_name,
                            'platform': 'whatsapp',
                            'name': name[:100] if len(name) > 3 else f'{source_name} - Grupo',
                            'url': href,
                            'invite_code': invite_code,
                            'state': detect_state(name),
                            'category': detect_category(name),
                            'scraped_at': datetime.now().isoformat()
                        })
                except Exception as e:
                    continue
            
            # Extrai links Telegram
            telegram_links = await page.query_selector_all('a[href*="t.me"]')
            print(f"   Encontrados {len(telegram_links)} links Telegram")
            
            for link in telegram_links:
                try:
                    href = await link.get_attribute('href')
                    name = await link.inner_text()
                    name = name.strip() if name else 'Sem nome'
                    
                    invite_code = extract_telegram_code(href)
                    if invite_code:
                        groups.append({
                            'source': source_name,
                            'platform': 'telegram',
                            'name': name[:100] if len(name) > 3 else f'{source_name} - Grupo',
                            'url': href,
                            'invite_code': invite_code,
                            'state': detect_state(name),
                            'category': detect_category(name),
                            'scraped_at': datetime.now().isoformat()
                        })
                except:
                    continue
                    
        except Exception as e:
            print(f"   ❌ Erro: {e}")
        finally:
            await browser.close()
    
    return groups

print("✅ Crawler Playwright Async carregado")

✅ Crawler Playwright Async carregado


## 4. Crawler específico para Quero Plantão (ASYNC)

In [5]:
async def crawl_quero_plantao_async():
    """Crawler ASYNC específico para queroplantao.com.br/grupos
    
    Este site tem tabs por estado que precisam ser clicadas.
    """
    
    groups = []
    url = 'https://queroplantao.com.br/grupos/'
    
    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(
            user_agent='Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
        )
        page = await context.new_page()
        
        try:
            print("   Acessando Quero Plantão...")
            await page.goto(url, timeout=60000)
            await page.wait_for_timeout(5000)
            
            # Fecha popup se existir
            try:
                await page.click('text=Aceito', timeout=3000)
            except:
                pass
            try:
                close_buttons = await page.query_selector_all('[class*="close"], .close, button[aria-label="Close"]')
                for btn in close_buttons:
                    try:
                        await btn.click(timeout=1000)
                    except:
                        pass
            except:
                pass
            
            # Lista de estados para tentar clicar
            estados = [
                'Brasil', 'São Paulo', 'Rio de Janeiro', 'Minas Gerais', 'Bahia',
                'Paraná', 'Rio Grande do Sul', 'Santa Catarina', 'Goiás',
                'Distrito Federal', 'Ceará', 'Pernambuco', 'Pará', 'Amazonas',
                'Maranhão', 'Paraíba', 'Rio Grande do Norte', 'Alagoas', 'Sergipe',
                'Piauí', 'Espírito Santo', 'Mato Grosso', 'Mato Grosso do Sul',
                'Rondônia', 'Acre', 'Amapá', 'Roraima', 'Tocantins'
            ]
            
            # Scroll inicial
            for _ in range(3):
                await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
                await page.wait_for_timeout(1000)
            
            # Tenta clicar em cada aba de estado
            for estado in estados:
                try:
                    # Tenta clicar no texto do estado
                    element = await page.query_selector(f'text="{estado}"')
                    if element:
                        await element.click(timeout=2000)
                        await page.wait_for_timeout(1500)
                except:
                    continue
            
            # Scroll final
            for _ in range(3):
                await page.evaluate('window.scrollTo(0, document.body.scrollHeight)')
                await page.wait_for_timeout(1000)
            
            # Extrai todos os links WhatsApp
            whatsapp_links = await page.query_selector_all('a[href*="chat.whatsapp.com"]')
            print(f"   Encontrados {len(whatsapp_links)} links WhatsApp")
            
            for link in whatsapp_links:
                try:
                    href = await link.get_attribute('href')
                    name = await link.inner_text()
                    name = name.strip() if name else ''
                    
                    if not name or len(name) < 3:
                        # Tenta pegar texto do elemento pai
                        try:
                            parent_text = await link.evaluate('el => el.parentElement?.innerText || ""')
                            name = parent_text.strip()[:100] if parent_text else 'Quero Plantão - Grupo'
                        except:
                            name = 'Quero Plantão - Grupo'
                    
                    invite_code = extract_whatsapp_code(href)
                    if invite_code:
                        groups.append({
                            'source': 'quero_plantao',
                            'platform': 'whatsapp',
                            'name': name[:100],
                            'url': href,
                            'invite_code': invite_code,
                            'state': detect_state(name),
                            'category': detect_category(name),
                            'scraped_at': datetime.now().isoformat()
                        })
                except:
                    continue
            
            # Telegram
            telegram_links = await page.query_selector_all('a[href*="t.me"]')
            print(f"   Encontrados {len(telegram_links)} links Telegram")
            
            for link in telegram_links:
                try:
                    href = await link.get_attribute('href')
                    name = await link.inner_text()
                    name = name.strip() if name else 'Quero Plantão - Telegram'
                    
                    invite_code = extract_telegram_code(href)
                    if invite_code:
                        groups.append({
                            'source': 'quero_plantao',
                            'platform': 'telegram',
                            'name': name[:100],
                            'url': href,
                            'invite_code': invite_code,
                            'state': detect_state(name),
                            'category': detect_category(name),
                            'scraped_at': datetime.now().isoformat()
                        })
                except:
                    continue
                    
        except Exception as e:
            print(f"   ❌ Erro: {e}")
        finally:
            await browser.close()
    
    return groups

print("✅ Crawler Quero Plantão Async carregado")

✅ Crawler Quero Plantão Async carregado


## 5. Executar Crawlers

In [6]:
async def run_all_crawlers():
    """Executa todos os crawlers e retorna os grupos"""
    
    all_groups = []
    
    print("="*70)
    print("   INICIANDO CRAWLER INDUSTRIAL")
    print("="*70)
    
    # ============================================
    # SITES COM HTML SIMPLES (requests)
    # ============================================
    
    print(f"\n🔍 ESCALA_PLANTAO")
    print(f"   URL: https://escaladeplantao.com.br/grupos")
    groups = crawl_with_requests('https://escaladeplantao.com.br/grupos', 'escala_plantao')
    all_groups.extend(groups)
    print(f"   ✅ {len(groups)} grupos extraídos")
    
    print(f"\n🔍 GRUPOS_MEDICOS")
    print(f"   URL: https://gruposmedicos.com.br/")
    groups = crawl_with_requests('https://gruposmedicos.com.br/', 'grupos_medicos')
    all_groups.extend(groups)
    print(f"   ✅ {len(groups)} grupos extraídos")
    
    # ============================================
    # SITES COM JAVASCRIPT (Playwright Async)
    # ============================================
    
    print(f"\n🔍 QUERO_PLANTAO")
    print(f"   URL: https://queroplantao.com.br/grupos/")
    try:
        groups = await crawl_quero_plantao_async()
        all_groups.extend(groups)
        print(f"   ✅ {len(groups)} grupos extraídos")
    except Exception as e:
        print(f"   ❌ Erro: {e}")
    
    print(f"\n🔍 SEARCH_PLANTAO")
    print(f"   URL: https://web.searchplantao.com.br/grupos")
    try:
        groups = await crawl_with_playwright_async(
            'https://web.searchplantao.com.br/grupos', 
            'search_plantao',
            wait_time=8000
        )
        all_groups.extend(groups)
        print(f"   ✅ {len(groups)} grupos extraídos")
    except Exception as e:
        print(f"   ❌ Erro: {e}")
    
    print(f"\n🔍 PLANTOES_BRASIL")
    print(f"   URL: https://plantoesmedicosbrasil.com.br/")
    try:
        groups = await crawl_with_playwright_async(
            'https://plantoesmedicosbrasil.com.br/', 
            'plantoes_brasil',
            wait_time=8000
        )
        all_groups.extend(groups)
        print(f"   ✅ {len(groups)} grupos extraídos")
    except Exception as e:
        print(f"   ❌ Erro: {e}")
    
    print(f"\n{'='*70}")
    print(f"   TOTAL BRUTO: {len(all_groups)} grupos")
    print("="*70)
    
    return all_groups

# Executa os crawlers
all_groups = asyncio.get_event_loop().run_until_complete(run_all_crawlers())

   INICIANDO CRAWLER INDUSTRIAL

🔍 ESCALA_PLANTAO
   URL: https://escaladeplantao.com.br/grupos
   ✅ 49 grupos extraídos

🔍 GRUPOS_MEDICOS
   URL: https://gruposmedicos.com.br/
   ✅ 322 grupos extraídos

🔍 QUERO_PLANTAO
   URL: https://queroplantao.com.br/grupos/
   Acessando Quero Plantão...
   Encontrados 0 links WhatsApp
   Encontrados 0 links Telegram
   ✅ 0 grupos extraídos

🔍 SEARCH_PLANTAO
   URL: https://web.searchplantao.com.br/grupos
   Acessando https://web.searchplantao.com.br/grupos...
   Encontrados 0 links WhatsApp
   Encontrados 0 links Telegram
   ✅ 0 grupos extraídos

🔍 PLANTOES_BRASIL
   URL: https://plantoesmedicosbrasil.com.br/
   Acessando https://plantoesmedicosbrasil.com.br/...
   Encontrados 0 links WhatsApp
   Encontrados 1 links Telegram
   ✅ 1 grupos extraídos

   TOTAL BRUTO: 372 grupos


## 6. Processar e Deduplica

In [ ]:
# Cria DataFrame
df = pd.DataFrame(all_groups)

if len(df) == 0:
    print("❌ Nenhum grupo encontrado!")
else:
    # Remove duplicatas por invite_code
    antes = len(df)
    df = df.drop_duplicates(subset=['invite_code'], keep='first')
    depois = len(df)
    
    print(f"\n📊 DEDUPLICAÇÃO:")
    print(f"   Antes: {antes} grupos")
    print(f"   Depois: {depois} grupos")
    print(f"   Removidos: {antes - depois} duplicatas")

In [ ]:
# Estatísticas
if len(df) > 0:
    print("\n" + "="*70)
    print("   ESTATÍSTICAS FINAIS")
    print("="*70)
    
    print(f"\n📊 POR FONTE:")
    print(df['source'].value_counts().to_string())
    
    print(f"\n📊 POR PLATAFORMA:")
    print(df['platform'].value_counts().to_string())
    
    print(f"\n📊 POR ESTADO (top 15):")
    print(df['state'].value_counts().head(15).to_string())
    
    print(f"\n📊 POR CATEGORIA (top 15):")
    print(df['category'].value_counts().head(15).to_string())

## 7. Filtrar Grupos Médicos

In [ ]:
# Categorias que NÃO são de médicos
categorias_nao_medicas = [
    'enfermagem', 'odontologia', 'fisioterapia', 'psicologia', 
    'nutricao', 'fonoaudiologia', 'assistencia_social', 'bombeiro',
    'socorrista', 'tecnico_enfermagem', 'educacao_fisica', 'farmacia'
]

# Filtra apenas médicos
if len(df) > 0:
    df_medicos = df[~df['category'].isin(categorias_nao_medicas)].copy()
    
    print(f"\n🩺 GRUPOS DE MÉDICOS: {len(df_medicos)} de {len(df)} total")
    print(f"\nCategorias incluídas:")
    print(df_medicos['category'].value_counts().to_string())
else:
    df_medicos = pd.DataFrame()

## 8. Exportar CSVs

In [ ]:
if len(df) > 0:
    # Salva todos os grupos
    df.to_csv('grupos_whatsapp_saude_COMPLETO.csv', index=False)
    print(f"✅ Salvo: grupos_whatsapp_saude_COMPLETO.csv ({len(df)} grupos)")
    
    # Salva apenas médicos
    df_medicos.to_csv('grupos_whatsapp_medicos_COMPLETO.csv', index=False)
    print(f"✅ Salvo: grupos_whatsapp_medicos_COMPLETO.csv ({len(df_medicos)} grupos)")
    
    # Salva por estado (útil para operação)
    estados_brasil = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 
                      'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN',
                      'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']
    df_por_estado = df_medicos[df_medicos['state'].isin(estados_brasil)]
    df_por_estado.to_csv('grupos_whatsapp_medicos_POR_ESTADO.csv', index=False)
    print(f"✅ Salvo: grupos_whatsapp_medicos_POR_ESTADO.csv ({len(df_por_estado)} grupos)")
    
    # Salva apenas WhatsApp (para Evolution API)
    df_whatsapp = df_medicos[df_medicos['platform'] == 'whatsapp']
    df_whatsapp.to_csv('grupos_whatsapp_EVOLUTION_API.csv', index=False)
    print(f"✅ Salvo: grupos_whatsapp_EVOLUTION_API.csv ({len(df_whatsapp)} grupos)")
else:
    print("❌ Nenhum grupo para salvar")

## 9. Visualizar Amostra

In [ ]:
if len(df_medicos) > 0:
    # Mostra amostra por estado
    print("\n" + "="*70)
    print("   AMOSTRA DE GRUPOS POR ESTADO")
    print("="*70)
    
    for state in ['SP', 'RJ', 'MG', 'BA', 'PR', 'RS', 'DF']:
        state_groups = df_medicos[df_medicos['state'] == state]
        if len(state_groups) > 0:
            print(f"\n📍 {state}: {len(state_groups)} grupos")
            for _, row in state_groups.head(3).iterrows():
                print(f"   • {row['name'][:50]}")
                print(f"     Código: {row['invite_code']}")

In [ ]:
if len(df_medicos) > 0:
    # Mostra amostra por especialidade
    print("\n" + "="*70)
    print("   AMOSTRA DE GRUPOS POR ESPECIALIDADE")
    print("="*70)
    
    for cat in ['cardiologia', 'pediatria', 'emergencia', 'ortopedia', 'anestesiologia']:
        cat_groups = df_medicos[df_medicos['category'] == cat]
        if len(cat_groups) > 0:
            print(f"\n🏥 {cat.upper()}: {len(cat_groups)} grupos")
            for _, row in cat_groups.head(2).iterrows():
                print(f"   • {row['name'][:50]}")

## 10. Resumo Final

In [ ]:
if len(df) > 0:
    df_whatsapp = df_medicos[df_medicos['platform'] == 'whatsapp'] if len(df_medicos) > 0 else pd.DataFrame()
    estados_brasil = ['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 
                      'MT', 'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN',
                      'RS', 'RO', 'RR', 'SC', 'SP', 'SE', 'TO']
    df_por_estado = df_medicos[df_medicos['state'].isin(estados_brasil)] if len(df_medicos) > 0 else pd.DataFrame()
    
    print("\n" + "="*70)
    print("   RESUMO FINAL")
    print("="*70)
    print(f"""
📊 GRUPOS EXTRAÍDOS:
   Total (todas áreas):     {len(df)} grupos
   Apenas médicos:          {len(df_medicos)} grupos
   Por estado:              {len(df_por_estado)} grupos
   WhatsApp (Evolution):    {len(df_whatsapp)} grupos

📁 ARQUIVOS GERADOS:
   • grupos_whatsapp_saude_COMPLETO.csv
   • grupos_whatsapp_medicos_COMPLETO.csv
   • grupos_whatsapp_medicos_POR_ESTADO.csv
   • grupos_whatsapp_EVOLUTION_API.csv

🚀 PRÓXIMO PASSO:
   Use o arquivo grupos_whatsapp_EVOLUTION_API.csv
   para integrar com a Evolution API e fazer a
   Júlia entrar nos grupos automaticamente.
""")
else:
    print("❌ Nenhum grupo foi extraído. Verifique os erros acima.")

## 11. Código de Integração com Evolution API

```python
import requests
import time
import pandas as pd

EVOLUTION_URL = "https://seu-evolution.com"
API_KEY = "sua-api-key"

def join_group(instance_name, invite_code):
    """Entra em um grupo de WhatsApp"""
    url = f"{EVOLUTION_URL}/group/acceptInvite/{instance_name}"
    
    response = requests.post(
        url,
        json={"inviteCode": invite_code},
        headers={"apikey": API_KEY}
    )
    return response.json()

# Lê o CSV
df = pd.read_csv('grupos_whatsapp_EVOLUTION_API.csv')

# Entra em cada grupo (com rate limiting)
for i, row in df.iterrows():
    result = join_group('julia-01', row['invite_code'])
    print(f"Grupo {row['name']}: {result}")
    
    # Máximo 5 grupos por dia por número
    if (i + 1) % 5 == 0:
        print("Limite diário atingido, trocar número ou aguardar")
        break
    
    time.sleep(60)  # 1 minuto entre cada entrada
```